In [23]:
##CoronaVirus predictor
import tensorflow as tf
from tensorflow import keras
import keras
import numpy as np
import os
import csv 

count = 70
size = 7
covidVec = np.empty((count, size, 1))
covidAns = np.empty((count, 1))

##hard coded, may obtain from CSV but it seems like an overhead
beginPolandIndex = 12388
endPolandIndex = 12456

table = {}
i = 0
with open('CaseStatistics.csv', newline = '', encoding = 'utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        table[i] = row
        i = i + 1
z = 0

for l in range(beginPolandIndex, endPolandIndex - 6):
    for j in range(7):
        covidVec[z, j, :] = float(table[l + j][4]) # dzienny przyrost 
    covidAns[z] = float(table[l + 7][4])
    z = z + 1

##normalization purpose
maxAnswer = np.amax(covidAns) / 2
##normalize
covidAns = covidAns / maxAnswer - 1
covidAns = covidAns[0:z]
maxval = np.amax(covidVec) / 2
covidVec = covidVec / maxAnswer - 1
covidVec = covidVec[0:z, :, :]

##Create net model
input = keras.engine.input_layer.Input(shape = (size,1), name = "input")

FlattenLayer = keras.layers.Flatten()

path = FlattenLayer(input)

for i in range(0,5):
    LayerDense1 = keras.layers.Dense(16, activation=None, use_bias=True, kernel_initializer='glorot_uniform')
    path = LayerDense1(path)
    LayerPReLU1 = keras.layers.PReLU(alpha_initializer='zeros', shared_axes=None)
    path = LayerPReLU1(path)

LayerDenseN = keras.layers.Dense(1,activation=None, use_bias=True, kernel_initializer='glorot_uniform')
output = LayerDenseN(path)

##---------------------------------
##Create tensorflow model
##---------------------------------
covidModel = keras.Model(input, output, name='covidEstimatior')

covidModel.summary() # Display summary

##Run training
rmsOptimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)#, epsilon=None, decay=0.0, amsgrad=False)

covidModel.compile(optimizer=rmsOptimizer,loss=keras.losses.mean_absolute_error)

covidModel.fit(covidVec, covidAns, epochs=300, batch_size=20, shuffle=True)
covidModel.save('covid.h5')


##get predicted result

covidVecW = covidVec[z-1:z,:,:]
covid = covidModel.predict(covidVecW)
print((covid[0]+1)*maxAnswer)

Model: "covidEstimatior"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 7, 1)              0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 7)                 0         
_________________________________________________________________
dense_175 (Dense)            (None, 16)                128       
_________________________________________________________________
p_re_lu_153 (PReLU)          (None, 16)                16        
_________________________________________________________________
dense_176 (Dense)            (None, 16)                272       
_________________________________________________________________
p_re_lu_154 (PReLU)          (None, 16)                16        
_________________________________________________________________
dense_177 (Dense)            (None, 16)            